# Prompt Templates, intro

## Colab-specific setup

Make sure you have a Database, a Secure Connect Bundle and a Database Token. See [DB Setup](https://cassio.org/db_setup/) on cassio.org for details, **paying attention to the notes for Colab users**.
Get ready to upload the Bundle and supply the Token string.

Likewise, ensure you have the necessary secret for the LLM provider of your choice: you'll be asked to input it shortly. See [API Setup](https://cassio.org/api_setup/) on cassio.org for details, **paying attention to the notes for Colab users**.

_Note: this colab is autogenerated from a [regular Jupyter notebook](https://cassio.org/frameworks/langchain/prompt-templates-basic/) hosted on cassio.org. Run all cells in this section to complete the setup before moving on to the demo content proper._

_Note: you can work with your own Cassandra cluster instead of Astra DB, provided it is reachable from the cloud: check the commented code below for the case `cqlMode="local"` and consult [cassio.org](https://cassio.org/frameworks/langchain/setup/#database-choice) for more details._

In [ ]:
# install required dependencies
! pip install \
    "git+https://github.com/hemidactylus/langchain@cassio#egg=langchain" \
    "git+https://github.com/datastax/python-driver.git@cep-vsearch#egg=cassandra-driver" \
    "cassio>=0.0.2" \
    "google-cloud-aiplatform>=1.25.0" \
    "jupyter>=1.0.0" \
    "openai==0.27.7" \
    "python-dotenv==1.0.0" \
    "tensorflow-cpu==2.12.0" \
    "tiktoken==0.4.0" \
    "transformers>=4.29.2"

In [ ]:
# Input your database keyspace name:
ASTRA_DB_KEYSPACE = input('Your Astra DB Keyspace name: ')

In [ ]:
# Input your Astra DB token string, the one starting with "AstraCS:..."
ASTRA_DB_APPLICATION_TOKEN = input('Your Astra DB Token: ')

In [ ]:
# Upload your Secure Connect Bundle zipfile:
import os
from google.colab import files


print('Please upload your Secure Connect Bundle')
uploaded = files.upload()
if uploaded:
    astraBundleFileTitle = list(uploaded.keys())[0]
    ASTRA_DB_SECURE_BUNDLE_PATH = os.path.join(os.getcwd(), astraBundleFileTitle)
else:
    raise ValueError(
        'Cannot proceed without Secure Connect Bundle. Please re-run the cell.'
    )

In [ ]:
# Set your secret(s) for LLM access (key names must match `providerValidator` in `llm_choice`)
llmProvider = 'OpenAI'  # 'VertexAI'
if llmProvider == 'OpenAI':
    apiSecret = input(f'Your secret for LLM provider "{llmProvider}": ')
    os.environ['OPENAI_API_KEY'] = apiSecret
elif llmProvider == 'VertexAI':
    # we need a json file
    print(f'Please upload your Service Account JSON for the LLM provider "{llmProvider}":')
    from google.colab import files
    uploaded = files.upload()
    if uploaded:
        vertexAIJsonFileTitle = list(uploaded.keys())[0]
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(os.getcwd(), vertexAIJsonFileTitle)
    else:
        raise ValueError(
            'No file uploaded. Please re-run the cell.'
        )
else:
    raise ValueError('Unknown/unsupported LLM Provider')

In [ ]:
# colab-specific override of helper functions
from cassandra.cluster import (
    Cluster,
)
from cassandra.auth import PlainTextAuthProvider

ASTRA_DB_CLIENT_ID = 'token'


def getCQLSession(mode='astra_db'):
    if mode == 'astra_db':
        cluster = Cluster(
            cloud={
                "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
            },
            auth_provider=PlainTextAuthProvider(
                ASTRA_DB_CLIENT_ID,
                ASTRA_DB_APPLICATION_TOKEN,
            ),
        )
        astraSession = cluster.connect()
        return astraSession
    # elif mode == 'local':
    #     cluster = Cluster(
    #         ['192.168.0.1', '192.168.0.2'],
    #         auth_provider=PlainTextAuthProvider(
    #             "username",
    #             "password!",
    #         ),
    #     )
    #     # See https://docs.datastax.com/en/developer/python-driver/latest/getting_started/#connecting-to-cassandra for more options
    #     localSession = cluster.connect()
    #     return localSession
    else:
        raise ValueError('Unknown CQL Session mode')

def getCQLKeyspace(mode='astra_db'):
    if mode == 'astra_db':
        return ASTRA_DB_KEYSPACE
    # elif mode == 'local':
    #     return <NAME_OF_YOUR_LOCAL_KEYSPACE>
    else:
        raise ValueError('Unknown CQL Session mode')

## Populate the Database

This notebook requires some data to be pre-populated on your database. Please follow these steps (roughly equivalent to [this section](https://cassio.org/db_setup/#import-sample-data) of the instructions on cassio.org):

In [ ]:
ASTRA_DB_KEYSPACE = getCQLKeyspace(mode="astra_db")

!curl https://raw.githubusercontent.com/CassioML/cassio-website/main/setup/provision_db/write_sample_data.cql --output write_sample_data.cql
!curl https://downloads.datastax.com/enterprise/cqlsh-astra-20230526-vectortype-bin.tar.gz --output cqlsh.tar.gz
!tar -xzf cqlsh.tar.gz
!./cqlsh-astra/bin/cqlsh \
    -b "$ASTRA_DB_SECURE_BUNDLE_PATH" \
    -u token \n    -p "$ASTRA_DB_APPLICATION_TOKEN" \
    -k "$ASTRA_DB_KEYSPACE" \
    -f write_sample_data.cql

### Colab preamble completed

The following cells constitute the demo notebook proper.

# Prompt Templates, intro

CassIO powers a sophisticated set of bindings to seamlessly inject data
from Cassandra tables into your LangChain prompt templates.

## Basic usage

In [1]:
from langchain.prompts import createCassandraPromptTemplate

This cell simply obtains a `Session` object, i.e. an active connection to your database. Replace with custom code if you are not using Astra DB.

In [2]:
# creation of the DB connection
cqlMode = 'astra_db'
session = getCQLSession(mode=cqlMode)
keyspace = getCQLKeyspace(mode=cqlMode)

In [3]:
ctemplate0 = """Please answer a question from a user.
Keep in mind that the user's age is {user_age} and they live in a city with
nickname {city_nickname}.

USER'S QUESTION: {user_question}
YOUR ANSWER:
"""

### Natural binding with the DB

In the (string) template above, some variables are to be filled with a DB lookup.

The following instructions specifies the details of the binding: for instance,
the variable `user_age` is to be found on table `people`, specifically in column `age`:

In [4]:
cassPrompt = createCassandraPromptTemplate(
    session=session,
    keyspace=keyspace,
    template=ctemplate0,
    input_variables=['city', 'name', 'user_question'],
    field_mapper={
        'user_age': ('people', 'age'),
        'city_nickname': ('nickname_by_city', 'nickname'),
    },
)

**Note** that in the command above we specify the _primary key columns_ as `input_variables`, and not the variable names found in the prompt string above.

When formatting the Prompt Template, we will have to specify the primary key values
for the DB lookup -- the rest is done by the prompt template.

In this case there are two lookups from as many tables: the prompt template
takes care of everything, provided you pass all the primary key columns required
across tables.

In [5]:
print(cassPrompt.format(city='turin', name='beppe',
                        user_question='Is functional programming fun?'))

Please answer a question from a user.
Keep in mind that the user's age is 2 and they live in a city with
nickname CereaNeh.

USER'S QUESTION: Is functional programming fun?
YOUR ANSWER:

